# Implementation of baseline using Artificial Neural Network with Original + Win-loss + feature vectors feature set and LLPD

Necessary files: logistic_regression_functions.py, ann_functions.py, testing_functions.py, win_loss_functions.py, feature_vectors_functions, LLPD_17.csv, LLPD_18.csv, LLPD_19.csv, A_LLPD.csv, H_LLPD.csv, A_LLPD_before_threshold.csv, H_LLPD_before_threshold.csv

In [ ]:
import logistic_regression_functions
import ann_functions
import testing_functions
import win_loss_functions
import feature_vectors_functions
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tqdm import tqdm

The baseline is implemented according to the paper An Improved Prediction System for Football a Match Result bu C. P. Igiri and E. O. Nwachukwu from 2014. The original features as described in the paper are used and the model is evaluated on the LLPD dataset.

The authors used only one season for training, so I did the same. I used seasons 2016/17 for training season 2017/18 as validation set and the second half of season 2018/19 for testing. The validation set is used to select the best checkpoint of the training according to the validation accuracy.

In [ ]:
X_train_win_loss, y_train = win_loss_functions.create_data(['LLPD_17.csv'], skip_rounds = 6, return_names=True)
results_val_win_loss, matches_per_round = win_loss_functions.create_data_single('LLPD_18.csv', ['LLPD_17.csv'], skip_rounds = 6, return_names=True)
# Dates are returned as well for dividing testing season into slices
results_test_win_loss, matches_per_round = win_loss_functions.create_data_single('LLPD_19.csv', ['LLPD_17.csv', 'LLPD_18.csv'],
                                return_dates=True, skip_rounds = 6, return_names=True)

Processing LLPD_17.csv season file.


In [ ]:
results_train_originals, matches_per_round = ann_functions.create_data_single('LLPD_17.csv', 53, 17,
                                   logistic_regression_functions.team_names_map_llpd,
                                   logistic_regression_functions.secondary_team_names_map_llpd,
                                   ternary_team_names_map=logistic_regression_functions.ternary_team_names_map_llpd)
results_val_originals, matches_per_round = ann_functions.create_data_single('LLPD_18.csv', 53, 18,
                                  logistic_regression_functions.team_names_map_llpd,
                                  logistic_regression_functions.secondary_team_names_map_llpd,
                                  ternary_team_names_map=logistic_regression_functions.ternary_team_names_map_llpd)
# Dates are returned as well for dividing testing season into slices
results_test_originals, matches_per_round = ann_functions.create_data_single('LLPD_19.csv', 53, 19,
                                  logistic_regression_functions.team_names_map_llpd,
                                  logistic_regression_functions.secondary_team_names_map_llpd,
                                  ternary_team_names_map=logistic_regression_functions.ternary_team_names_map_llpd,
                                  return_dates=True)
X_train_originals = results_train_originals.drop('FTR', axis=1)
y_train_originals = results_train_originals['FTR']

In [ ]:
# Concatenating the two feature sets together
X_train_mix = pd.concat([X_train_originals, X_train_win_loss], axis=1)
results_val_mix = pd.concat([results_val_originals, results_val_win_loss], axis=1)
results_test_mix = pd.concat([results_test_originals, results_test_win_loss], axis=1)
X_train_mix = X_train_mix.loc[:,~X_train_mix.columns.duplicated()].copy()
results_val_mix = results_val_mix.loc[:,~results_val_mix.columns.duplicated()].copy()
results_test_mix = results_test_mix.loc[:,~results_test_mix.columns.duplicated()].copy()

In [ ]:
A = pd.read_csv('A_LLPD.csv')
H = pd.read_csv('H_LLPD.csv')
A_before_threshold = pd.read_csv('A_LLPD_before_threshold.csv')
H_before_threshold = pd.read_csv('H_LLPD_before_threshold.csv')

In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

They didnt say what ANN architecture they used. I used 3 dense layers with 64, 32 and 16 neurons. I used dropout to overcome overfitting.

I always let the model train for 100 epochs and took the weights from the best epoch according to validation accuracy.

To use all the data available, the first half of the testing season was added to the training data.

In [ ]:
# Some rounds in the beginning are ignored, this is the correct index
# of the start of the second half of the season
start_test_index = 13 * matches_per_round

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 131ms/step - loss: 0.7347 - accuracy: 0.7857


  7%|▋         | 1/15 [00:17<04:01, 17.22s/it]

1/1 [==============================] - 0s 130ms/step - loss: 1.2408 - accuracy: 0.3636


 13%|█▎        | 2/15 [00:38<04:16, 19.71s/it]

...

1/1 [==============================] - 0s 128ms/step - loss: 0.8878 - accuracy: 0.7333


 93%|█████████▎| 14/15 [04:38<00:20, 20.46s/it]

1/1 [==============================] - 0s 127ms/step - loss: 1.3264 - accuracy: 0.4000


100%|██████████| 15/15 [05:00<00:00, 20.01s/it]


0.5263157973164007


The testing accuracy is 52.63%.

Experimenting with feature vectors extracted before applying the threshold.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['LLPD_17.csv'], skip_rounds = 6, return_names=True)

Processing LLPD_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H_before_threshold)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 139ms/step - loss: 0.7848 - accuracy: 0.6429


  7%|▋         | 1/15 [00:21<04:59, 21.38s/it]

1/1 [==============================] - 0s 147ms/step - loss: 1.1346 - accuracy: 0.5455


 13%|█▎        | 2/15 [00:39<04:13, 19.54s/it]

...

1/1 [==============================] - 0s 162ms/step - loss: 0.8221 - accuracy: 0.7333


 93%|█████████▎| 14/15 [05:28<00:31, 31.79s/it]

1/1 [==============================] - 0s 141ms/step - loss: 1.5173 - accuracy: 0.5000


100%|██████████| 15/15 [06:10<00:00, 24.71s/it]


0.5315789555248461


The testing accuracy is 53.16%.

Experimenting with feature vectors including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['LLPD_17.csv'], skip_rounds = 6, return_names=True)

Processing LLPD_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model1.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 148ms/step - loss: 0.7382 - accuracy: 0.7857


  7%|▋         | 1/15 [00:19<04:26, 19.04s/it]

1/1 [==============================] - 0s 147ms/step - loss: 1.3379 - accuracy: 0.6364


 13%|█▎        | 2/15 [00:38<04:08, 19.10s/it]

...

1/1 [==============================] - 0s 144ms/step - loss: 0.8651 - accuracy: 0.6667


 93%|█████████▎| 14/15 [05:32<00:25, 25.11s/it]

1/1 [==============================] - 0s 154ms/step - loss: 1.3664 - accuracy: 0.2000


100%|██████████| 15/15 [06:14<00:00, 24.97s/it]


0.5210526416176244


The testing accuracy is 52.11%.

Experimenting with feature vectors extracted before applying the threshold and including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['LLPD_17.csv'], skip_rounds = 6, return_names=True)

Processing LLPD_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 131ms/step - loss: 0.7129 - accuracy: 0.8571


  7%|▋         | 1/15 [00:18<04:19, 18.51s/it]

1/1 [==============================] - 0s 146ms/step - loss: 1.2385 - accuracy: 0.4545


 13%|█▎        | 2/15 [00:36<03:56, 18.20s/it]

...

1/1 [==============================] - 0s 159ms/step - loss: 0.7248 - accuracy: 0.7333


 93%|█████████▎| 14/15 [05:28<00:31, 31.79s/it]

1/1 [==============================] - 0s 130ms/step - loss: 1.4907 - accuracy: 0.5000


100%|██████████| 15/15 [05:50<00:00, 23.35s/it]


0.5210526427156047


The testing accuracy is 52.11%.